In [1]:
import pandas as pd
import glob
from google.cloud import bigquery
from google.cloud import storage
import os
import shutil
import re
import time
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import io 
from googleapiclient.http import MediaIoBaseDownload
import warnings
warnings.simplefilter('ignore')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Thomas/Google Drive/Projects/Google_code_n_Oauth/Client_Oauth/Google_auth/valid-pagoda-132423-c6ac84b41833.json"

In [2]:
def get_oauth(path, version):
    """
    get authorization to create create the spreadsheet
    Currently, version 4 of Google API
    """
    SCOPES = ['https://www.googleapis.com/auth/drive']
    path_token = path+"token.json"
    path_credential = path + "credentials.json"
    store = file.Storage(path_token)
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets(path_credential, SCOPES)
        creds = tools.run_flow(flow, store)
    if version ==4:
        service = build('sheets', 'v4', http=creds.authorize(Http()))
    else:
        service = build('drive', 'v3', http=creds.authorize(Http()))
    return service

In [3]:
def upload_extra_files(path_credential, path_to_save, dic_id):
    drive_service = get_oauth(path = path_credential, version = 3)
    for i in range(0,4):
        request = drive_service.files().export_media(fileId=dic_id['id'][i],
                                             mimeType='text/csv')
        destination = path_to_save + '/' + dic_id['name'][i]
        fh = io.FileIO(destination, mode='w')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))

In [4]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

In [5]:
def upload_blob(bucket_name, destination_blob_name,  source_file_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
          source_file_name,
          destination_blob_name))

In [6]:
def upload_bq(dataset_name, bucket_uri, name_table, sql_schema):
    client = bigquery.Client()
    dataset_ref = client.dataset(dataset_name)
    job_config = bigquery.LoadJobConfig()
    list_bq_schema = []
    for sql in sql_schema:
        list_bq_schema.append(bigquery.SchemaField(sql[0], sql[1]))
    #job_config.autodetect = True
    job_config.schema = list_bq_schema
    job_config.skip_leading_rows = 1
    # The source format defaults to CSV, so the line below is optional.
    job_config.source_format = bigquery.SourceFormat.CSV
    uri = bucket_uri
    load_job = client.load_table_from_uri(
    uri,
    dataset_ref.table(name_table),
    job_config = job_config)  # API request
    print('Starting job {}'.format(load_job.job_id))
    load_job.result()  # Waits for table load to complete.
    print('Job finished.')

In [7]:
def translate_files(old_path, path, new_year = 2001, previous_year = 2000):
    #year_to_change = str(int(year) - delta)
    #### Change path sh
    do_file = path + '/01_all_process.sh'
    s = open(do_file).read()
    s = s.replace(old_path, path)
    s = s.replace('YEAR=' + str(previous_year), 'YEAR=' + str(new_year))
    f = open(do_file, 'w')
    f.write(s)
    f.close()
    #### change do file less than 2007 
    do_file = path + '/inf_2007/02_create_int_files.do'
    s = open(do_file).read()
    s = s.replace('data'+str(previous_year), 'data'+str(new_year))
    s = s.replace('Date = "' + str(previous_year), 'Date = "' +str(new_year))
    s = s.replace('local y = ' + str(previous_year), 'local y = ' +str(new_year))
    s = s.replace(old_path, path)
    f = open(do_file, 'w')
    f.write(s)
    f.close()
    #### Change in do-file 02
    do_file = path + '/sup_2007/02_create_int_files.do'
    s = open(do_file).read()
    s = s.replace(old_path, path)
    s = s.replace('data' + str(previous_year), 'data' + str(new_year))
    s = s.replace('tradedata-' + str(previous_year), 'tradedata-' + str(new_year))
    f = open(do_file, 'w')
    f.write(s)
    f.close()
    #### Change in do-file 02
    do_file = path + '/03_translate.do'
    #s = open(do_file).read()
    with open(path + '/03_translate.do') as f:
        lines = f.readlines()
        if int(new_year) >= 2007:
            print(old_path+'/sup_2007')
            lines[0] = 'cd ' + path+'/sup_2007 \n'
        else:
            lines[0] = 'cd ' + path + '/inf_2007 \n'
    with open(path + '/03_translate.do', "w") as f:
        f.writelines(lines)

In [8]:
def bash_stata(path):
    translate_command = 'bash ' + path + '/01_all_process.sh'
    print('Begin translate', path+ '/01_translate.sh')
    os.system(translate_command)

In [9]:
def prepare_df_city_country(df, cities_path,
                            countries_path):
    """
    Extract cities name from city list
    Match only countries in the list of countries
    """
    
    # Cities
    list_cities = pd.read_csv(cities_path, index_col=None)
    # Extract cities
    raw_list = list_cities.city.tolist()
    df['city_prod'] = df['city'].str.extract(r"(?=(" + '|'.join(raw_list) +
                                             r"))")
    missing_cities_before = df['city_prod'].isnull().sum()
    print('Sum missing cities {}'.format(missing_cities_before))

    example_null = df[df['city_prod'].isnull()]['city'].unique()
    print('Exemple of missing cities {}'.format(example_null[:5]))

    list_variables_to_extract = ['address', 'Company_name']
    df['matching_city'] = 'City_var'
    for name in list_variables_to_extract:
        unmatch = df[df['city_prod'].isnull()]
        unmatch['retrieve_ciy'] = unmatch[name].str.extract(
        r"(?=(" + '|'.join(raw_list) + r"))")
        df.loc[unmatch['retrieve_ciy'].index, 'matching_city'] = name
        df['city_prod'] = df['city_prod'].fillna(unmatch['retrieve_ciy'])
        missing_cities_after = df['city_prod'].isnull().sum()
        perc_found = 1 - missing_cities_after / missing_cities_before
        count_retrieve = missing_cities_before - missing_cities_after
        print('{} cities have been retrieved .\n' \
          'Missing cities have now {:.1%} filled values. \n' \
          'We will try to retrieve {} missing cities from another \n' \
          ' list'.format(count_retrieve,
                     perc_found,
                     missing_cities_after))
    ### Retrieve city from list_random which is list of autonomous region or counties
    list_random_regex = ['省.+$', '内蒙古自治区.+$', '西藏自治区.+$', '新疆维吾尔.+$', '广西壮族.+$']
    unmatch_1 = df[df['city_prod'].isnull()]
    unmatch_1['city_2'] = unmatch_1['city'].str.extract(r"(" + '|'.join(list_random_regex) +")")

    list_random = ['省', '内蒙古自治区', '西藏自治区', '新疆维吾尔', '广西壮族']
    unmatch_1['city_2'] = unmatch_1['city_2'].str.replace("(" + '|'.join(list_random) +")", '', regex=True)
    #### List unique province
    raw_list_province = list_cities.province.unique().tolist()
    unmatch_1['province_'] = unmatch_1['city'].str.extract(r"(" + '|'.join(raw_list_province) +")")
    #### replace NaN with 其他
    unmatch_1['city_2'] = unmatch_1['city_2'].fillna('其他')
    df.loc[unmatch['retrieve_ciy'].index, 'matching_city'] = 'another_list'
    df['city_prod'] = df['city_prod'].fillna(unmatch_1['city_2'])
    missing_cities_after_unmatch_1 = df['city_prod'].isnull().sum()
    count_retrieve_list_2 = missing_cities_after - missing_cities_after_unmatch_1
    perc_found_1 = 1 - missing_cities_after_unmatch_1 / missing_cities_before
    print('{0} cities have been retrieved from another list. Those unmatched' \
          'in another list are filled qith "其他"\n' \
          'City has now {1:.0%} filled values. \nThere are {2} unknow cities'.format(
          count_retrieve_list_2, perc_found_1, missing_cities_after_unmatch_1))
    ### Drop 
    df = df.drop(columns=['Company_name', 'address', 'city'])
    print(df.isna().sum())
    df = df.dropna()
    #### Merge provinces
    df = pd.merge(
        df,
        list_cities,
        how='left',
        left_on=['city_prod'],
        right_on=['city'],
        suffixes=('', '_y'))

    df = df.drop(columns=['city'])
    df['province'] = df['province'].fillna(unmatch_1['province_'])
    df['province'] = df['province'].fillna('其他')

    print(df.groupby('imp_exp')['value'].sum())
    return df

In [10]:
def prepare_business_type_and_intermediate(df, business_path,
                                           intermediate_path):
    # retrieve business types
    business = pd.read_csv(business_path, index_col=None)
    df_nan_biz = df[df['Business_type'].isnull()]

    nan_source = df_nan_biz['Business_type'].isna().sum()
    print(
        'There are {} missing Business type in the Dataset'.format(nan_source))
    if nan_source !=0:
        raw_list = business.x.tolist()
        df_nan_biz['Business_type'] = df_nan_biz['Company_name'].str.extract(
            r"(?=(" + '|'.join(raw_list) + r"))")

        nan_after = df_nan_biz['Business_type'].isna().sum()

        perc_total = (1 - nan_after / nan_source)
        count_retrieve = nan_source - nan_after
        shape_df = len(df)
        print(
            'Business type is filled at {0:.0%} with {1}' \
            ' values retrieve from company name.\n The missing {2}' \
            ' are filling with "其他" \n The dataset has {3}' \
            ' rows'.format(perc_total,
                       count_retrieve,
                       nan_after, 
                       shape_df))
    # df_nan_biz = df_nan_biz.dropna()

        df = df.fillna(df_nan_biz)
        df['Business_type'] = df['Business_type'].fillna('其他')
        df['Business_type'] = np.where(df['Business_type'] == ' ', '其他',
                                   df['Business_type'])
    
    

    # Create intermediate firms
    intermediate = pd.read_csv(intermediate_path, index_col=None)
    raw_list = intermediate.intermediate.tolist()
    df['intermediate'] = df['Company_name'].str.extract(
        r"(?=(" + '|'.join(raw_list) + r"))")
    df['intermediate'] = df[[
        'intermediate'
    ]].applymap(lambda x: 'No' if pd.isnull(x) else 'Yes')

    print(df.groupby('intermediate')['intermediate'].count())
    #df = df.drop(columns= ['Company_name'])
    return df

In [11]:
def save_to_gz(path, name, business_path, intermediate_path, cities_path,
               countries_path):
    allFiles = glob.glob(path + "/*.csv")
    list_ = []

    list_to_keep = [
        'Date',
        'ID',
        'city',
        'address',
        'Company_name',
        'Business_type',
        'Trade_type',
        'imp_exp',
        'HS',
        'Origin_and_destination',
        'value',
        'Quantity',
    ]

    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0, usecols=list_to_keep)
        list_.append(df)
        
    df = pd.concat(list_, axis=0, ignore_index=True)

    df = df[list_to_keep]
    df = df[(df['imp_exp'] == '进口') | (df['imp_exp'] == '出口') ]
    #### Print total import/export raw data
    print(df.groupby('imp_exp')['value'].sum())
    print(df.shape)

    df_1 = prepare_business_type_and_intermediate(df, business_path,
                                                  intermediate_path)

    df_1 = prepare_df_city_country(df_1, cities_path, countries_path)

    #### Keep var
    #### Rename
    #df_1 = df_1.rename(index=str, columns={"origin_id": "Country_id"})

    #### To avoid any types problem, change type manually
    df_1['Date'] = df_1['Date'].astype('int64')
    df_1['ID'] = pd.to_numeric(df_1['ID'], errors='coerce')
    rem = df_1['ID'].isna().sum()
    print('{} rows have been removed from ID due to wrong formating'.format(rem))
    df_1 = df_1[pd.notnull(df_1['ID'])]
    df_1['ID'] = df_1['ID'].astype('int64')
    #df_1['Company_name'] = df_1['Company_name'].astype('object')
    df_1['Business_type'] = df_1['Business_type'].astype('object')
    df_1['intermediate'] = df_1['intermediate'].astype('object')
    df_1['Trade_type'] = df_1['Trade_type'].astype('object')
    df_1['province'] = df_1['province'].astype('object')
    df_1['city_prod'] = df_1['city_prod'].astype('object')
    df_1['matching_city'] = df_1['matching_city'].astype('object')
    df_1['imp_exp'] = df_1['imp_exp'].astype('object')
    df_1['HS'] = df_1['HS'].astype('int64')
    df_1['Origin_and_destination'] = df_1['Origin_and_destination'].astype(
        'object')
    #df_1['Country_id'] = df_1['Country_id'].astype('int64')
    df_1['value'] = df_1['value'].astype('int64')
    df_1['Quantity'] = df_1['Quantity'].astype('int64')
    
    list_aggregate = [
        'Date', 'ID','Business_type', 'intermediate', 'Trade_type', 'province', 'city_prod','matching_city',
        'imp_exp', 'HS', 'Origin_and_destination'
    ]
    list_to_keep = ['Date', 'ID','Business_type', 'intermediate', 'Trade_type', 'province', 'city_prod','matching_city',
        'imp_exp', 'HS', 'Origin_and_destination', 'value', 'Quantity']
    df_1 = df_1.groupby(list_aggregate).sum().reset_index()
    df_1= df_1[list_to_keep]
    #### Unique HS
    unique_HS = df_1.groupby(['imp_exp'])['HS'].nunique()

    #### Unique Country
    unique_country = df_1.groupby(
        ['imp_exp'])['Origin_and_destination'].nunique()

    print('Unique HS \n {} \n unique countries \n {}'.format(
        unique_HS, unique_country))
    
    print(df_1.groupby('imp_exp')['value'].sum())

    df_1.to_csv(
        path + "/" + name,
        sep=',',
        header=True,
        index=False,
        chunksize=100000,
        compression='gzip',
        encoding='utf-8')

    return df_1

In [12]:
def compressed_gz(path_file_name, file_name, bucket_name, blob_name,
                  dataset_name, SQL_schema,business_path, intermediate_path, cities_path, countries_path):
    """
    function to Compress into gz and move to the GCP + BQ
    path_file_name: path where the csv are and where the future gz will be stored
    file_name: Name of the gz to save
    bucket_name: Backet name to store the data in GCS. Need to be root
    blob_name: Name of the subfolder in the bucket
    dataset_name: Name given to the dataset stored in BG
    SQL_schema: Manually define SQL schema
    cities_path: Path to open the cities list
    countries_path: Path to open the countries list
    """
    
    #### Open all cv
    df = save_to_gz(path_file_name,
                    file_name,
                    business_path,
                    intermediate_path,
                    cities_path,
                    countries_path)
    ##### Remove csv
    #os.remove(file_name)
    
    #### Move to GCS
    print('Begin to upload to GCS')
    destination_blob_name = blob_name + '/' + file_name
    source_file_name = path_file_name + '/' + file_name
    upload_blob(bucket_name = bucket_name,
            destination_blob_name = destination_blob_name,
            source_file_name = source_file_name)
    
    #### Move to BG
    print('Begin to move to BG')
    bucket_uri = 'gs://' + bucket_name + '/' + blob_name + '/'+ file_name
    name_table = re.findall(r"^\w+", file_name)[0]
    upload_bq(dataset_name = dataset_name, 
          bucket_uri = bucket_uri,
          name_table = name_table,
          sql_schema = SQL_schema)
    
    ##### Remove gz
    os.remove(source_file_name)

# Upload extra file

We upload the list_intermediate, list_country, list_cities, business from Google Drive. 

- Filename is the rar name

## Attention

To run the program, the root folder needs to have:

- a subfolder `translate` that contain the do files to translate the dta
- a subfolder `extra` to host the extra file downloaded from the Drive



In [13]:
old_path = '/Users/Thomas/Downloads/Stata_translate'
path = '/Users/Thomas/Downloads/Stata_translate'
path_source = path + '/' +'data_' + str(2007)
path_save_extra = path + '/extra'
##### Need to be set dynamically
file_name = 'data_2007.rar'
cities_path = '/Users/Thomas/Downloads/Stata_translate/extra/list_cities.csv'
countries_path = '/Users/Thomas/Downloads/Stata_translate/extra/list_country.csv'
business_path = '/Users/Thomas/Downloads/Stata_translate/extra/business.csv'
intermediate_path = '/Users/Thomas/Downloads/Stata_translate/extra/list_intermediate_firms.csv'

In [ ]:
path_credential = '/Users/Thomas/Google Drive/Projects/Google_code_n_Oauth/Client_Oauth/Google_auth/'
dic_id = {
    'id' : ['1jSjy6eCqT1ogHsRQH268thmy24HpydftpydRBL715lY',
           '1UsjJZ-Fveujx4nvRXGE1Q1l7nbHzqxK0tBfPSdqq3iU',
           '1i8u2h8Oh5KyDQwo6aT1km_S03topF4lRnRV9fasl51c', 
           '1DY5tCtMMw2kcbcbHI30Zo1-QYb6NmR_ZDEqQURKCc4s'],
    'name': ['list_intermediate_firms.csv', 'list_country.csv',
    'list_cities.csv', 'business.csv']
}
upload_extra_files(path_credential  = path_credential,
                   path_to_save = path_save_extra, 
                   dic_id = dic_id)

# Upload files from GCS

It is better to update all the files so that we can proceed in batch

We need to set filename and source blob

In [ ]:
download_blob(bucket_name = 'store_data_trade',
            source_blob_name = 'raw_data/data2008.rar',
            destination_file_name = path + '/' + file_name)

# Translate the data

We can translate after having downloaded the dta

1. First we need to change the year in the do files and sh file
2. Translate dta

In [ ]:
translate_files(old_path, path, new_year = 2000, previous_year = 2001)

In [ ]:
bash_stata(path)

# Prepare Dataset and Move to the cloud

The next step prepares the dataset:

- retrieve business types
- Create intermediate firms
- Merge Cities
- Merge provinces
- Merge countries
- Aggreate the data 

The final dataset has the following variables

- 'Date'
- 'ID'
- 'Company_name'
- 'Business_type'
- 'Trade_type'
- 'city_prod'
- 'province'
- 'imp_exp'
- 'HS'
- 'Origin_and_destination'
- 'Country_id'
- 'value'
- 'Quantity'

After that, the data is moved to Google Cloud Storage and Big Query

In [14]:
SQL_schema = [
['Date','INTEGER'],
['ID','INTEGER'],
['Business_type','STRING'],
['intermediate', 'STRING'],    
['Trade_type','STRING'],
['province','STRING'],   
['city_prod','STRING'],
['matching_city','STRING'],
['imp_exp','STRING'],
['HS','INTEGER'],
['Origin_or_destination','STRING'],
['value','INTEGER'],
['Quantity','INTEGER']]


In [ ]:
file_name_cloud = "data_2005.gz"
compressed_gz(path_file_name= path,
              file_name = file_name_cloud,
              bucket_name = 'store_data_trade',
              blob_name = 'gz_files',
              dataset_name = 'trade_data',
              SQL_schema = SQL_schema, 
              business_path = business_path,
              intermediate_path = intermediate_path,
              cities_path = cities_path,
              countries_path = countries_path)

# Batch mode

For a batch mode, we can run the following function

In [15]:
import time
def batch_model(list_year, download = True):
    for i in list_year: 
        
        start = time.time()
        file_name = 'data_' + str(i) + '.rar'
        #### This is for year after 2007
        path_source = path + '/' +'data_' + str(i)
        source_blob_name = 'raw_data/data'+ str(i) + '.rar'
        print(file_name, '\n',
        source_blob_name)
    
        if download:
            download_blob(bucket_name = 'store_data_trade',
                source_blob_name = source_blob_name,
                destination_file_name = path + '/' + file_name)
        previous_year  = int(i) - 1
        if int(i) > 2000:
            translate_files(old_path, path, new_year = i, previous_year = previous_year)
    
        bash_stata(path)
    
        file_name_cloud = 'data_' + str(i) + '.gz'
        try:
            compressed_gz(path_file_name= path,
              file_name = file_name_cloud,
              bucket_name = 'store_data_trade',
              blob_name = 'gz_files',
              dataset_name = 'trade_data',
              SQL_schema = SQL_schema, 
              business_path = business_path,
              intermediate_path = intermediate_path,
              cities_path = cities_path,
              countries_path = countries_path)
            end = time.time()
            print('Year {} finished in {} minutes'.format(i, (end - start)/60))
            
            #### Remove csv
            allFiles = glob.glob(path + "/*.csv")
            for i in allFiles: 
                os.remove(i) 
            
        except:
            allFiles = glob.glob(path + "/*.csv")
            for i in allFiles: 
                os.remove(i)
                
    f_end = time.time()
    print((f_end - start)/360)

In [16]:
list_year  = range(2003, 2011)

batch_model(list_year, download = True)


data_2003.rar 
 raw_data/data2003.rar
Blob raw_data/data2003.rar downloaded to /Users/Thomas/Downloads/Stata_translate/data_2003.rar.
Begin translate /Users/Thomas/Downloads/Stata_translate/01_translate.sh
imp_exp
出口    4.371436e+11
进口    4.096980e+11
Name: value, dtype: float64
(14771186, 12)
There are 0 missing Business type in the Dataset
intermediate
No     9671127
Yes    5100059
Name: intermediate, dtype: int64
Sum missing cities 706459
Exemple of missing cities ['内蒙古自治区呼伦贝尔盟' '山东省其它' '河北省其它' '内蒙古自治区哲里木盟' '甘肃省其它']
428504 cities have been retrieved .
Missing cities have now 60.7% filled values. 
We will try to retrieve 277955 missing cities from another 
 list
630639 cities have been retrieved .
Missing cities have now 89.3% filled values. 
We will try to retrieve 75820 missing cities from another 
 list
75820 cities have been retrieved from another list. Those unmatchedin another list are filled qith "其他"
City has now 100% filled values. 
There are 0 unknow cities
Date            

Blob raw_data/data2007.rar downloaded to /Users/Thomas/Downloads/Stata_translate/data_2007.rar.
/Users/Thomas/Downloads/Stata_translate/sup_2007
Begin translate /Users/Thomas/Downloads/Stata_translate/01_translate.sh
imp_exp
出口    1.216709e+12
进口    9.524580e+11
Name: value, dtype: float64
(10491595, 12)
There are 1136841 missing Business type in the Dataset
Business type is filled at 83% with 942152 values retrieve from company name.
 The missing 194689 are filling with "其他" 
 The dataset has 10491595 rows
intermediate
No     6230369
Yes    4261226
Name: intermediate, dtype: int64
Sum missing cities 496912
Exemple of missing cities ['广东省番禹' '辽宁省沉阳' '安徽省宣城' '浙江省其它' '广西壮族其它']
221919 cities have been retrieved .
Missing cities have now 44.7% filled values. 
We will try to retrieve 274993 missing cities from another 
 list
446145 cities have been retrieved .
Missing cities have now 89.8% filled values. 
We will try to retrieve 50767 missing cities from another 
 list
50767 cities have bee

imp_exp
出口    1573111692574
进口    1374977620152
Name: value, dtype: int64
Begin to upload to GCS
File /Users/Thomas/Downloads/Stata_translate/data_2010.gz uploaded to gz_files/data_2010.gz.
Begin to move to BG
Starting job 65e41828-dc38-49fc-8f78-d9e5470975d6
Job finished.
Year 2010 finished in 50.94630013306936 minutes
8.491316455602647


Job finished in 494 minutes, so 8 hours. Partly due to slow connection

# Download .gz locally

In [ ]:
list_year  = range(2000, 2011)
for i in list_year: 
    file_name = 'data_' + str(i) + '.gz'
    source_blob_name = 'gz_files/data_'+ str(i) + '.gz'
    print(file_name, '\n',
        source_blob_name)
    download_blob(bucket_name = 'store_data_trade',
                source_blob_name = source_blob_name,
                destination_file_name =file_name)